# NumPy `ndarray` Basics - SOLUTIONS

## Table of Contents

1. [The De Facto Standard for Array Data](#1.-The-De-Facto-Standard-for-Array-Data)
2. [Anatomy of an `ndarray`: Structure and Memory](#2.-Anatomy-of-an-`ndarray`:-Structure-and-Memory)
3. [Array Creation and Logical Views (Views vs. Copies)](#3.-Array-Creation-and-Logical-Views-(Views-vs.-Copies))
4. [Aggregations and Axes](#4.-Aggregations-and-Axes)
5. [Broadcasting: The "Stretch" Rule](#5.-Broadcasting:-The-"Stretch"-Rule)
6. [Why Vectorize? The Speed Advantage](#6.-Why-Vectorize?-The-Speed-Advantage)

## 1. The De Facto Standard for Array Data

NumPy is the foundational library for High Performance Computing (HPC) and Machine Learning (ML) in Python. Libraries like PyTorch, Pandas, and Scikit-learn are built upon or mirror the NumPy API. Learning NumPy is essential for mastering the Array Programming paradigm.

NumPy provides the `ndarray` (N-dimensional array), a powerful, high-performance, and uniform container that enables highly efficient memory management, indexing, slicing, and, most importantly, vectorized arithmetic.

In [1]:
import numpy as np

## 2. Anatomy of an `ndarray`: Structure and Memory

Unlike a standard Python list, an `ndarray` is a fixed-size, structured block of contiguous memory. Its efficiency comes from these four key, immutable properties:

- **Data**: A pointer to the memory location holding the elements.
- **dtype**: The data type (e.g., `int32`, `float64`) which is uniform across all elements.
- **Shape**: A tuple defining the size along each dimension (e.g., $(100, 50)$ for 100 rows and 50 columns).
- **Strides**: The number of bytes to step in memory to reach the next element along each dimension—this is how NumPy efficiently handles different shapes and views.

Let's explore these properties by creating a large dataset.

---

**Quick Docs**
- `np.arange(start, stop, step)`: Returns evenly spaced values in the half-open interval $[\text{start}, \text{stop})$.
- `arr.nbytes`: Total bytes consumed by the array's elements (in bytes).
- `arr.ndim`: The number of array dimensions (integer).
- `arr.size`: The total number of elements in the array (integer).
- `arr.shape`: The tuple of array dimensions.


In [ ]:
# Use a large number to clearly demonstrate the memory density of ndarrays
N = 50_000_000

In [ ]:
# SOLUTION: Create the input data array with the numbers 1 to 50_000_000 (inclusive).
# np.arange generates values within a half-open interval [start, stop), so we use N + 1 as the stop value.
arr = np.arange(1, N + 1)

In [ ]:
# SOLUTION: Calculate how large the array is in GB with nbytes.
# GB is 1e9 bytes. The .nbytes attribute returns the total bytes consumed by the elements.
# Note: This demonstrates that arrays are dense memory blocks, unlike pointer-heavy Python lists.
arr.nbytes / 1e9

In [ ]:
# SOLUTION: How many dimensions does the array have? (ndim)
arr.ndim

In [ ]:
# SOLUTION: How many elements does the array have? (size)
arr.size

In [ ]:
# SOLUTION: What is the shape of the array?
arr.shape

## 3. Array Creation and Logical Views (Views vs. Copies)

Arrays can logically represent data in many ways (e.g., 1D signal, 2D image, 4D video batch) independent of the underlying physical memory block.

A critical performance feature is that operations like transposing or `reshape` often return a **View** instead of a **Copy**. A View only changes the metadata (`shape` and `strides`) without duplicating the physical data, making these operations nearly instantaneous.

---

**Quick Docs**
- `np.linspace(start, stop, num)`: Returns `num` evenly spaced samples, calculated over the interval $[\text{start}, \text{stop}]$.
- `np.random.default_rng().random(size)`: Returns random floats in $[0.0, 1.0)$. `size` can be a tuple.
- `arr.sort()`: Sorts an array in-place (modifies the original data). Use `np.sort(arr)` to return a sorted copy.
- `arr.reshape(new_shape)`: Returns a View with a new shape. One dimension can be -1, instructing NumPy to calculate the size automatically.
- `np.resize(arr, new_shape)`: Returns a new array with the specified shape. If the new shape is larger, it fills the new elements by repeating the original array.


In [ ]:
# SOLUTION: Create a new array with 5_000_000 elements containing equally spaced values between 0 to 1000 (inclusive).
# np.linspace returns 'num' evenly spaced samples over the closed interval [start, stop].
arr = np.linspace(0, 1000, 5_000_000)
arr

In [ ]:
# SOLUTION: Create a random array that is 10_000 rows by 5_000 columns.
# np.random.default_rng().random(size) returns random floats in [0.0, 1.0). size can be a tuple.
arr = np.random.default_rng().random((10_000, 5_000))
arr

In [ ]:
# SOLUTION: Sort that array (in-place).
# Note: arr.sort() modifies the array directly, which is typically faster than creating a copy.
arr.sort()

In [ ]:
# SOLUTION: Reshape the array to have the last dimension of length 5. 
# Using -1 lets NumPy automatically calculate the first dimension.
# .reshape() returns a View (not a copy) when possible, so no data is duplicated.
arr_new = arr.reshape(-1, 5)
arr_new

## 4. Aggregations and Axes

When performing aggregations (like `sum`, `mean`, `max`), you must specify the **Axis** you want to collapse (or reduce) the array along.

- **Axis 0**: The first dimension (often rows in 2D). Aggregating across Axis 0 produces a result for each column.
- **Axis 1**: The second dimension (often columns in 2D). Aggregating across Axis 1 produces a result for each row.

---

**Quick Docs**
- `np.sum(a, axis=None)`: Sum of array elements over a given axis.
  - `axis=0`: Collapse the rows (sum vertical columns).
  - `axis=1`: Collapse the columns (sum horizontal rows).


In [ ]:
# SOLUTION: Find the sum of each row in the reshaped array (arr_new) above.
# To sum the row's content, we reduce across the columns (axis=1).
# axis=1 collapses the second dimension (columns), leaving one sum per row.
arr_sum = np.sum(arr_new, axis=1)
arr_sum

## 5. Broadcasting: The "Stretch" Rule

Broadcasting is NumPy's mechanism for performing arithmetic between arrays of different shapes. If dimensions don't match, NumPy attempts to "stretch" the smaller array to match the larger one.

**The Compatibility Rule:** Two dimensions are compatible when:
1. They are equal, or
2. One of them is 1.

If a dimension is 1, NumPy logically copies that single value across the dimension to match the other array's shape **without allocating any new memory**.

---

**Quick Docs**
- **Arithmetic Operators** (`/`, `*`, `+`, `-`): These operate element-wise. Broadcasting occurs if shapes are different but compatible.
- `np.allclose(a, b)`: Returns `True` if two floating-point arrays are element-wise equal within a tolerance. Essential for comparisons instead of using `==`.


In [ ]:
# SOLUTION: Normalize each row of the 2D array (arr_new) by dividing by the sum you just computed (arr_sum).
# 'arr_new' has shape (M, N) and 'arr_sum' has shape (M,).
# To successfully divide, we reshape 'arr_sum' to (M, 1) so broadcasting can stretch it across the N columns.
# Alternative approaches: arr_sum[:, np.newaxis] or arr_sum[:, None] also work.
arr_normalized = arr_new / arr_sum.reshape(-1, 1)
arr_normalized

In [ ]:
# SOLUTION (EXTRA CREDIT): Prove that your normalized array is actually normalized.
# If normalized correctly, the sum of every row should now be 1.0.
# We compute row sums (axis=1) and check if they are all close to 1.0 using np.allclose.
np.allclose(np.sum(arr_normalized, axis=1), 1.0)

## 6. Why Vectorize? The Speed Advantage

The entire Array Programming paradigm hinges on **Vectorization**.

Why use complex shapes and broadcasting instead of simple Python `for` loops?

NumPy's array functions are implemented in highly optimized native code (C/C++, Fortran). An operation like `A + A**2`, where `A` is a massive `ndarray`, is often $\mathbf{100\times}$ faster than performing the equivalent element-wise operation using explicit Python loops.

**Always choose a vectorized NumPy function or operator over a manual Python loop.**